# 34、Swin Transformer论文精讲及其PyTorch逐行实现

In [4]:
from typing import Optional
import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F

## 1、如何从图像获取embedding
###  使用 unfold 分块

- 基于pytorch `unfold`的API来将图片进行分块，也就是模仿卷积的思路，设置`kernel_size=stride=patch_size`，得到分块后的图片
- 得到格式为`[bs, num_patch, patch_depth]`的张量
- 将张量与形状为`[patch_depth, model_dim_C]`的权重矩阵进行乘法操作，即可得到形状为`[bs, num_patch, model_dim_C]`的patch embedding

###  使用卷积

- `patch_depth`是等于`input_channel*patch_size*patch_size`
- `model_din_C`相当于二维卷积的输出通道数目
- 将形状为`[patch_depth, model_dim_C]`的权重矩阵转换为`[model_dim_C, input_channel, patch_size, patch_size]`的卷积核
- 调用PyTorch的`conv2d` API得到卷积的输出张量,形状为`[bs, output_channel, height, width]`
- 转换为`[bs, num_patch, model_dim_C]`的格式,即为`patch embedding`

In [ ]:
# 分别使用 unfold 和卷积操作实现 image->embedding 操作
def image2emb_naive(img: Tensor, weight, patch_size=16):
    assert img.dim() == 4
    batch_size, channel, img_width, img_height = img.shape
    # shape: batch_size, patched, patch_num
    region = F.unfold(img, patch_size, stride=patch_size)
    region.transpose_(-1, -2)
    print(region.shape)
    return region @ weight


def image2emb_conv(img: Tensor, kernel, patch_size=16):
    """基于二维卷积实现patch embedding,embedding 维度就是卷积通道数"""
    conv = F.conv2d(img, kernel, stride=patch_size)
    bs, oc, ow, oh = conv.shape
    patch_emb = conv.reshape((bs, oc, ow*oh)).transpose(-1, -2) # (bs, ow*oh, oc)
    return patch_emb


## 2、如何构建MHSA并计算其复杂度？

- 基于输入x进行三个映射分别得到q,k,v
    -   此步复杂度为$3LC^2$，其中L为序列长度，C为特征大小
-   将q,k,v拆分成多头的形式，注意这里的多头各自计算不影响，所以可以与bs维度进行统一看待
-   计算$qk^T$，并考虑可能的掩码，即让无效的两两位置之间的能量为负无穷，掩码是在shift window MHSA中会需要，而在window MHSA中暂不需要
    -   此步复杂度为$L^2C$
-   计算概率值与v的乘积
    -   此步复杂度为$L^2C$
-   对输出进行再次映射
    -   此步复杂度为$LC^2$
-   总体复杂度为$4LC^2 + 2L^2C$